# set up blank model

In [2]:
import json
target_model_config = {
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": None,
  "eos_token_id": 2,
  "finetuning_task": "glue:rte",
  "gradient_checkpointing": False,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 15,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.0",
  "type_vocab_size": 1,
  "use_cache": True,
  "vocab_size": 50265
}

from transformers.models.roberta.modeling_tf_roberta import TFRobertaForSequenceClassification
from transformers import RobertaConfig

new_config = RobertaConfig(**target_model_config)
blank_model = TFRobertaForSequenceClassification(new_config)
blank_model.build()



/Users/briancruz/2023-fall-cs-194-294-merging-llms/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blank_model.save_pretrained('blank_model')

# define weave pattern

In [3]:
import random
from random import randint
model_weaving_config = {
    # The task (i.e. the classification head should match the task at hand)
    "target_model_template": "./blank_model",
    # layer assignments
    "layer_assignments": [
        {
            "type": "SingleLayer",
            "params": {
                # Pick one model randomly, p=0.5
                "donor": ["textattack/roberta-base-MNLI",
                          "textattack/roberta-base-RTE"][randint(0, 1)],
                # Pick a layer within [i-1,i+1], keeping it between 0 and 11
                "hidden_layer_number": min(11, max(0, randint(i - 1,i + 1))),
            },
        } for i in range(15)
    ],
}

model_weaving_config

{'target_model_template': './blank_model',
 'layer_assignments': [{'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-RTE',
    'hidden_layer_number': 0}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-MNLI',
    'hidden_layer_number': 0}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-MNLI',
    'hidden_layer_number': 2}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-MNLI',
    'hidden_layer_number': 2}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-MNLI',
    'hidden_layer_number': 5}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-RTE',
    'hidden_layer_number': 5}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-RTE',
    'hidden_layer_number': 6}},
  {'type': 'SingleLayer',
   'params': {'donor': 'textattack/roberta-base-MNLI',
    'hidden_layer_number': 8}},
  {'type': 'SingleLayer',
   'param

# Weave

In [4]:
from llm_weaver.weave import weave_models

weaved_model = weave_models(target_model_template=blank_model, layer_assignments=model_weaving_config["layer_assignments"])

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [5]:
weaved_model.save_pretrained('weaved_model')

# Evaluate

??